## Image classifier

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualisation tool
import tensorflow as tf #deep learning library
from PIL import Image # Image handling
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

Using TensorFlow backend.
Using TensorFlow backend.


### Helper functions

In [12]:
def create_output(lst, name):
    output_set['label'] = lst
    output_set.to_csv(path_or_buf={name}.csv, index=False)
    return

def image_to_rgb_matrix(path, pooling = False, size=4):
    """
    This function transfers an image to 3D-matrix containing per
    pixel the value of the RGB-values. This is needed as input for
    the CNN's.
    """
    new_width = 256
    new_height = 256
    
    img = Image.open(path)
    width, height = img.size
    
    # Get required paddings for cropping
    left = (width - new_width)/2
    top = (height - new_height)/2
    right = (width + new_width)/2
    bottom = (height + new_height)/2
    
    #Crop the center of the image
    img = img.crop((left, top, right, bottom))
    
    arr = np.array(img)/255
    
    if pooling:
        arr = pooling_images(arr, size=size)
    
    return arr


def multiple_images_to_matrix(last, first = 1, path_input = 'train_set/train_set', pooling=False, size=4):    
    matrices = []
    shapes = []
    for i in range(first, last + 1):
        path = path_input+f'/train_{i}.jpg'
        matrix = image_to_rgb_matrix(path, pooling=pooling, size=size)
        matrices.append(matrix)
        shapes.append(matrix.shape)
        
    return (matrices, shapes)

def multiple_images_to_matrix_test(last, first = 1, path_input = 'test_set/test_set', pooling=False, size=4):    
    matrices = []
    shapes = []
    for i in range(first, last + 1):
        path = path_input+f'/test_{i}.jpg'
        matrix = image_to_rgb_matrix(path, pooling=pooling, size=size)
        matrices.append(matrix)
        shapes.append(matrix.shape)
    return (matrices, shapes)

def pooling_images(image, size=4, kind='max'):
    height, width, depth = image.shape
    output = np.zeros([height//size, width//size, depth])
    for i in range(0, height, size):
        for j in range(0, width, 2):
            for k in range(depth):
                batch = image[i:i+size, j:j+size, k]
                if kind == 'max':
                    output[i//size, j//size, k] = batch.max()
                elif kind == 'mean':
                    output[i//size, j//size, k] = batch.mean()
    return output

#### Builder

In [ ]:
# First step is to get X and y data
x_train, shape = multiple_images_to_matrix(15000, 1, pooling=True)
x_train = np.stack(x_train, axis=0)
y_train = keras.utils.to_categorical(np.array(pd.read_csv('train_labels.csv')['label'])[:15000] -1, num_classes=80)

# x_val, shape = multiple_images_to_matrix(550, 501, pooling=True)
# x_val = np.stack(x_val, axis=0)
# y_val = keras.utils.to_categorical(np.array(pd.read_csv('train_labels.csv')['label'])[500:550] -1, num_classes=80)

x_test, shape = multiple_images_to_matrix_test(7653, 1, pooling=True)
x_test = np.stack(x_test, axis=0)


In [ ]:
# Then the model has to be built up
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(64,64,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(80, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [ ]:
# Next step is compiling and fitting the model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(x_train, y_train, batch_size =32, epochs=50, verbose=1, validation_split=0.001)
score = model.evaluate(x_val, y_val, batch_size=32)

In [ ]:
# And applying the model to predict the classes of test data
y_prob = model.predict(x_test)
y_classes = y_prob.argmax(axis=-1) + 1
y_classes

In [ ]:
# And then the output has to be copied in the sample

import datetime
import string
time = str(datetime.datetime.now())
time = ''.join(ch for ch in time if ch.isalnum())
output_set = pd.read_csv('sample.csv')
output_set['label'] = 71

if y_classes.shape[0] == output_set.shape[0]:
    output_set['label'] = y_classes
else:
    for i in range(y_classes.shape[0]):
        output_set['label'].iloc[i] = y_classes[i]
output_set.to_csv(path_or_buf=f'sample{time}.csv', index=False)

In [3]:
import datetime
import string

def fit_to_csv (model, x_train, x_test):
    y_prob = model.predict(x_test)
    y_classes = y_prob.argmax(axis=-1) + 1

    time = str(datetime.datetime.now())
    time = ''.join(ch for ch in time if ch.isalnum())
    output_set = pd.read_csv('sample.csv')
    output_set['label'] = 71
    
    if y_classes.shape[0] == output_set.shape[0]:
        output_set['label'] = y_classes
    else:
        for i in range(y_classes.shape[0]):
            output_set['label'].iloc[i] = y_classes[i]
    output_set.to_csv(path_or_buf=f'sample{time}.csv', index=False)
    return

    

## Resnet50 builder
The next step is to use the Resnet50 pre-trained model

In [4]:
#Helper functions

import os
import numpy as np
import tensorflow as tf
import h5py
import math

def load_dataset():
    train_dataset = h5py.File('datasets/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('datasets/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))
    
    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples) (m, Hi, Wi, Ci)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples) (m, n_y)
    mini_batch_size - size of the mini-batches, integer
    seed -- this is only for the purpose of grading, so that you're "random minibatches are the same as ours.
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[0]                  # number of training examples
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[permutation,:,:,:]
    shuffled_Y = Y[permutation,:]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:,:,:]
        mini_batch_Y = shuffled_Y[k * mini_batch_size : k * mini_batch_size + mini_batch_size,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[num_complete_minibatches * mini_batch_size : m,:,:,:]
        mini_batch_Y = shuffled_Y[num_complete_minibatches * mini_batch_size : m,:]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)].T
    return Y


def forward_propagation_for_predict(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters
    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    W3 = parameters['W3']
    b3 = parameters['b3'] 
                                                           # Numpy Equivalents:
    Z1 = tf.add(tf.matmul(W1, X), b1)                      # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                    # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2, A1), b2)                     # Z2 = np.dot(W2, a1) + b2
    A2 = tf.nn.relu(Z2)                                    # A2 = relu(Z2)
    Z3 = tf.add(tf.matmul(W3, A2), b3)                     # Z3 = np.dot(W3,Z2) + b3
    
    return Z3

def predict(X, parameters):
    
    W1 = tf.convert_to_tensor(parameters["W1"])
    b1 = tf.convert_to_tensor(parameters["b1"])
    W2 = tf.convert_to_tensor(parameters["W2"])
    b2 = tf.convert_to_tensor(parameters["b2"])
    W3 = tf.convert_to_tensor(parameters["W3"])
    b3 = tf.convert_to_tensor(parameters["b3"])
    
    params = {"W1": W1,
              "b1": b1,
              "W2": W2,
              "b2": b2,
              "W3": W3,
              "b3": b3}
    
    x = tf.placeholder("float", [12288, 1])
    
    z3 = forward_propagation_for_predict(x, params)
    p = tf.argmax(z3)
    
    sess = tf.Session()
    prediction = sess.run(p, feed_dict = {x: X})
        
    return prediction

In [5]:
# Imports

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
from urllib.request import urlopen,urlretrieve
from PIL import Image
from tqdm import tqdm_notebook
%matplotlib inline
from sklearn.utils import shuffle
# import cv2
# from resnets_utils import *

from keras.models import load_model
from sklearn.datasets import load_files   
from keras.utils import np_utils
from glob import glob
from keras import applications
from keras.preprocessing.image import ImageDataGenerator 
from keras import optimizers
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from keras.callbacks import TensorBoard,ReduceLROnPlateau,ModelCheckpoint

### Without pre-training

In [ ]:
img_height,img_width = 64,64 
num_classes = 80
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions = Dense(num_classes, activation= 'softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [ ]:
from keras.optimizers import SGD, Adam
# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(x_train, y_train, epochs = 2, batch_size = 64)
model.fit(x_train, y_train, epochs = 1, batch_size = 64, validation_split = 0.1, verbose = 0)

In [ ]:
# And applying the model to predict the classes of test data
y_prob = model.predict(x_test)
y_classes = y_prob.argmax(axis=-1) + 1
y_classes

In [ ]:
# And then the output has to be copied in the sample

import datetime
import string
time = str(datetime.datetime.now())
time = ''.join(ch for ch in time if ch.isalnum())
output_set = pd.read_csv('sample.csv')
output_set['label'] = 71

if y_classes.shape[0] == output_set.shape[0]:
    output_set['label'] = y_classes
else:
    for i in range(y_classes.shape[0]):
        output_set['label'].iloc[i] = y_classes[i]
output_set.to_csv(path_or_buf=f'sample{time}.csv', index=False)

### With pre-training

In [21]:
# First step is to get X and y data
x_train_pt, shape = multiple_images_to_matrix(1000, 1, pooling=True, size=2)
print(1)
x_train_pt = np.stack(x_train_pt, axis=0)
print(2)
y_train_pt = keras.utils.to_categorical(np.array(pd.read_csv('train_labels.csv')['label'])[:1000] -1, num_classes=80)
print(3)

# x_val, shape = multiple_images_to_matrix(550, 501, pooling=True)
# x_val = np.stack(x_val, axis=0)
# y_val = keras.utils.to_categorical(np.array(pd.read_csv('train_labels.csv')['label'])[500:550] -1, num_classes=80)


# x_test_pt, shape = multiple_images_to_matrix_test(100, 1, pooling=True, size=2)
# print(4)
# x_test_pt = np.stack(x_test_pt, axis=0)


11

2
2
33

4
4


In [45]:
x_test_pt = load_pooled_images_test(1500)

100100

200200

300300

400
400
500500

600
600
700700

800800

900900

1000
1000
1100
1100
1200
1200
1300
1300
1400
1400
15001500



In [7]:
img_height_pt,img_width_pt = 128,128
num_classes = 80
base_model_pt = applications.resnet50.ResNet50(weights= None, include_top=False, input_shape= (img_height_pt,img_width_pt,3))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Colocations handled automatically by placer.


C:\Users\Corne\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '
C:\Users\Corne\Anaconda3\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [8]:
x = base_model_pt.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.7)(x)
predictions_pt = Dense(num_classes, activation= 'softmax')(x)
model_pt = Model(inputs = base_model_pt.input, outputs = predictions_pt)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
from keras.optimizers import SGD, Adam

# sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
adam = Adam(lr=0.0001)
model_pt.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

es = EarlyStopping(monitor='val_acc', mode='min', verbose=1)

In [47]:
# model.fit(x_train, y_train, epochs = 2, batch_size = 64)
history = model_pt.fit(x_train_pt, y_train_pt, epochs = 5, batch_size = 32, validation_split = 0.1, verbose = 1, callbacks=[es])

Train on 900 samples, validate on 100 samplesTrain on 900 samples, validate on 100 samples

Epoch 1/5Epoch 1/5



KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
# fit_to_csv(model_pt, x_train_pt, x_test_pt)
fit_to_csv(history, x_train_pt, x_test_pt)

### Miscellaneous

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
model.summary()

In [49]:
def multiple_images_to_matrix_pooled_train(last = 30612, first = 1, path_input = 'train_set/train_set', pooling=True, size=2):    
    for i in range(first, last + 1):
        path = path_input+f'/train_{i}.jpg'
        matrix = image_to_rgb_matrix(path, pooling=pooling, size=size)
        path_out = path_input+'/pooled2'+f'/train_{i}'
        np.save(path_out, matrix)
    return

def multiple_images_to_matrix_pooled_test(last = 7653, first = 1, path_input = 'test_set/test_set', pooling=True, size=2):    
    for i in range(first, last + 1):
        path = path_input+f'/test_{i}.jpg'
        matrix = image_to_rgb_matrix(path, pooling=pooling, size=size)
        path_out = path_input+'/pooled2'+f'/test_{i}'
        np.save(path_out, matrix)
    return

def load_pooled_images_train(last=30612, first=1, path_input = 'train_set/train_set/pooled2'):
    matrices = []
    for i in range(first, last+1):
        matrices.append(np.load(path_input+f'/train_{i}.npy'))
    return np.stack(matrices, axis=0)

def load_pooled_images_test(last=7653, first=1, path_input = 'test_set/test_set/pooled2'):
    matrices = []
    for i in range(first, last+1):
        matrices.append(np.load(path_input+f'/test_{i}.npy'))
        if i % 100 == 0:
            print(i)
    return np.stack(matrices, axis=0)


In [ ]:
multiple_images_to_matrix_pooled_train()

In [38]:
multiple_images_to_matrix_pooled_test(first=207)

KeyboardInterrupt: 

KeyboardInterrupt: 

In [ ]:
# Early stopping
# Model Checkpoint
# Dataset vergroten (augmentation) --> ImageGenerator
# Shortcuts toeveogen

# Meer training data
# Meer layers toeveogen
# Resoution toevoegen
# Mean pooling ipv max pooling
# Grayscale ipv kleur?

# Resnet50 van Keras laden
# Googlenet


## To Do


### Data/preprocessing
- grayscale ipv kleur
- hogere resolutie (85 x 85 of 128 x 128)
- mean pooling / pixel linksboven ipv max pooling
- Volledige training set

### Models
- Resnet50 van Keras 
- Pretrained Resnet50 van Keras
- Googlenet
- Layers toevoegen aan beste model?
- Shortcuts toevoegen aan basis model
- Model checkpoint
- Early stoppping

### Tweaks na model compilation
- ImageDataGenerator toevoegen
